In [1]:
from TorchMetricLogger import TorchMetricLogger as TML
from TorchMetricLogger import TmlMetric, TmlLoss, TmlMetricFunction, TMLBinaryAccuracy, TMLDiceCoefficient
import torch

In [2]:
tml = TML()

p_1d = torch.ones((10, 5))
y_1d = torch.ones((10, 5))

p_1d[:, 3:] = 0

p_2d = torch.ones((10, 5, 5))
y_2d = torch.ones((10, 5, 5))

p_2d[0] = 0

w = torch.ones(10)
w[0] = 0.5

tml.log(
    meow=TmlMetric(p_1d, y_1d, metric_class=TMLBinaryAccuracy), 
    miau=TmlMetric(p_1d, y_1d, metric_class=TMLDiceCoefficient),
    wuff=TmlMetric(p_2d, y_2d, weights=w, metric_class=TMLDiceCoefficient)
)

In [3]:
tml.metrics["meow"].reduce()

0.6

In [4]:
tml.metrics["wuff"].partial

[[array([0.03846154, 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ],
        dtype=float32),
  array([0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ], dtype=float32)]]

In [5]:
tml.metrics["wuff"].reduce()

0.94939274

In [6]:
tml.metrics["miau"].reduce()

0.7777777910232544

In [7]:
from TorchMetricLogger import TorchMetricLogger as TML
from TorchMetricLogger import TmlMetric, TmlLoss, TmlMetricFunction, TMLBinaryAccuracy, TMLDiceCoefficient
import torch
tml = TML()

labels = torch.ones(100)
predictions = torch.zeros(100)
weights = torch.ones(100)

TmlMetric(labels, predictions, weights=weights, metric_class=TMLBinaryAccuracy)

metrics_weights = TMLBinaryAccuracy()(TmlMetric)

AttributeError: type object 'TmlMetric' has no attribute 'gold_labels'